In [1]:
import os
import re
import sqlite3
from collections import Counter

from tqdm import tqdm

import html2text
from sites import Seek, Jora

In [2]:
connection = sqlite3.connect('jobs.db')
cursor = connection.cursor()

In [3]:
seek = Seek(None)
jora = Jora(None)

In [4]:
jobs = cursor.execute('SELECT * FROM jobs').fetchall()
old_jobs = [str(x[0]) for x in cursor.execute('SELECT id FROM corpus').fetchall()]
jobs = [x for x in jobs if str(x[0]) not in old_jobs]
for job in tqdm(jobs):
    if job[6] == 'seek':
        content = seek.get_job_description(job[0])
    elif job[6] == 'jora':
        content = jora.get_job_description(job[0])
    else:
        continue
    if content is None:
        cursor.execute(f'INSERT INTO corpus VALUES ({str(job[0])}, true)', )
    else:
        with open('corpus/' + str(job[0]), 'w+') as f:
            content = html2text.html2text(content)
            f.write(content)
            cursor.execute(f'INSERT INTO corpus VALUES ({str(job[0])}, true)', )
    connection.commit()

  0%|          | 0/1324 [00:00<?, ?it/s]


RuntimeError: This event loop is already running

In [ ]:
files = os.listdir('corpus')
counter = Counter()
for file in tqdm(files):
    with open('corpus/' + str(file), 'r') as f:
        content = f.read()
        result = [x for x in re.split(r'\s', content) if x != '']
        counter.update(result)
counter.most_common(100)

In [ ]:
files = os.listdir('corpus')
double_counter = Counter()
for file in tqdm(files):
    with open('corpus/' + str(file), 'r') as f:
        content = f.read()
        result = [x for x in re.split(r'\s', content) if x != '']
        for i, word in enumerate(result):
            try:
                combo = word + ' ' + result[i + 1] + ' ' + result[i + 2]
                double_counter[combo] += 1
            except IndexError:
                pass
double_counter.most_common(100)

In [3]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [5]:
queue = []

jobs = cursor.execute("SELECT id, title, company FROM jobs").fetchall()
with tqdm(total=len(jobs)) as pbar:
    for i, job_source in enumerate(jobs):
        for j in range(i + 1, len(jobs)):
            job_target = jobs[j]
            title_score = similar(job_source[1], job_target[1])
            company_score = similar(job_source[2], job_target[2])
            queue.append(((job_source, job_target), (title_score, company_score)))
        pbar.update(1)
        
queue = sorted(queue, key=lambda x: x[1], reverse=True)

100%|██████████| 2415/2415 [02:48<00:00, 14.35it/s] 


In [19]:
nine = [x for x in queue if x[1][0] > 0.9 and x[1][1] > 0.9 and x[1][0] != 1 and x[1][1] != 1]

In [20]:
len(nine)

3

In [24]:
for job in nine:
    print(job[0][0][1], job[0][0][2])
    print(job[0][1][1], job[0][1][2])
    print(job[1])
    print()

Software Engineer (C#/ WPF) RPMGlobal
Software Engineer (C#/WPF) RPM Global
(0.9811320754716981, 0.9473684210526315)

Software Engineer (C#/ WPF) RPMGlobal
Software Engineer (C#/WPF) RPM Global
(0.9811320754716981, 0.9473684210526315)

Software Engineer (C# / C++) RPMGlobal
Software Engineer (C#/C++) RPM Global
(0.9629629629629629, 0.9473684210526315)


In [10]:
new_jobs = cursor.execute("SELECT id, title, company FROM jobs").fetchall()
jobs = cursor.execute("SELECT id, title, company FROM jobs").fetchall()
duplicates = []
for (id_source, title_source, company_source) in tqdm(new_jobs):
    temp_duplicates = []
    for (id_target, title_target, company_target) in jobs:
        if id_source == id_target: continue
        title_score = SequenceMatcher(None, title_source, title_target).ratio()
        company_score = SequenceMatcher(None, company_source, company_target).ratio()
        if title_score > 0.9 and company_score > 0.9:
            temp_duplicates.append(id_target)
    duplicates.append((id_source, temp_duplicates))
duplicates = [x for x in duplicates if len(x[1]) != 0]
# for id_source, temp_duplicates in duplicates:
#     cursor.execute(f"UPDATE jobs SET duplicate='{','.join(temp_duplicates)}' WHERE id='{id_source}'")
#     connection.commit()
print('Duplicates found:', len(duplicates))

100%|██████████| 2415/2415 [14:49<00:00,  2.72it/s]

Duplicates found: 1083


In [15]:
for id_source, temp_duplicates in duplicates:
    source_result = cursor.execute("SELECT id, title, company FROM jobs WHERE id = ?", (id_source, )).fetchone()
    print(source_result)
    for id_target in temp_duplicates:
        print(cursor.execute("SELECT id, title, company FROM jobs WHERE id = ?", (id_target, )).fetchone())
    print()

(74406376, 'Junior Software Engineer - Clipchamp', 'Microsoft')
('Software-Engineer-6e7be500a12e86b3b61dcc8dcf84e081', 'Junior Software Engineer - Clipchamp', 'Microsoft')
('Software-Engineer-b7d130f48a646806122fe6b21448d790', 'Junior Software Engineer - Clipchamp', 'Microsoft')
('Software-Engineer-c19328f107ea75c85136dc355d4cd22d', 'Junior Software Engineer - Clipchamp', 'Microsoft')

(74235982, 'Software Developer', 'Software Projects')
('Software-Developer-cd924d9f4902f0625087b3234d6e5f69', 'Software Developer', 'Software Projects')

(74419917, 'Web Developer', 'INTELICS Pty Ltd')
('Web-Developer-2c6e221cc4675132bd0d75aee8516fb5', 'Web Developer', 'INTELICS Pty Ltd')

(74399025, 'Principal Software Developer', 'Queensland Police Service')
('Software-Developer-6c6705079b30bc8a42916546ae9d8280', 'Principal Software Developer', 'Queensland Police Service')
('Software-Developer-c41cc06ea051210b97a72e7d15e8d35e', 'Principal Software Developer', 'Queensland Police Service')

(74350655, 'J

In [18]:
for id_source, temp_duplicates in tqdm(duplicates):
    dups = ','.join([str(x) for x in temp_duplicates])
    cursor.execute(f"UPDATE jobs SET duplicate='{dups}' WHERE id='{str(id_source)}'")
    connection.commit()

100%|██████████| 1083/1083 [00:20<00:00, 53.79it/s]


In [24]:
def get_duplicate_status(id_source):
    status = []
    for id_target in cursor.execute(f"SELECT duplicate FROM jobs WHERE id='{id_source}'").fetchone()[0].split(','):
        status.append(cursor.execute(f"SELECT status FROM jobs WHERE id='{id_target}'").fetchone()[0])
    if 'applied' in status:
        return 'applied'
    if 'easy_filter' in status:
        return 'easy_filter'
    if 'interested_read' in status:
        return 'interested_read'
    if 'interested' in status:
        return 'interested'
    if all([x=='not_interested' for x in status]):
        return 'not_interested'
    return '???'

not_interested
not_interested
not_interested
not_interested
not_interested
